In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [ ]:
train = pd.read_csv(data_path+"train.csv" )

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from collections import defaultdict

In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from collections import defaultdict

# 1. KoBERT 모델 및 토크나이저 로드 (tiktoken 대신 기본 BertTokenizer 사용)
tokenizer = BertTokenizer.from_pretrained("skt/kobert-base-v1")
model = BertModel.from_pretrained("skt/kobert-base-v1")

# 2. 불용어 리스트
stopwords = ['##는', '##을', '##습니다', '.', '##이', '##가', '저', '그', '이', '을']

# 3. 키워드 추출 함수 정의
def extract_keywords(text, top_n=10):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokens = tokenizer.tokenize(text)

    with torch.no_grad():
        outputs = model(**inputs)
        attentions = outputs.attentions[0]
        attention_mean = attentions.mean(dim=1).squeeze()
        token_scores = attention_mean.diagonal()

    token_score_pairs = [(token, score.item()) for token, score in zip(tokens, token_scores)]
    keywords = [(token, score) for token, score in token_score_pairs 
                if token not in stopwords and not token.startswith('##')]
    top_keywords = sorted(keywords, key=lambda x: x[1], reverse=True)[:top_n]
    return [keyword for keyword, score in top_keywords]

# 4. train 데이터 로드 (필요 시 경로 수정)
# 예: train = pd.read_csv('your_data.csv')

# 5. '인적사고'로 그룹화하고 키워드 추출
category_keywords = defaultdict(list)
for category, group in train.groupby('인적사고'):
    combined_text = " ".join(group['text'].astype(str).tolist())
    keywords = extract_keywords(combined_text, top_n=10)
    category_keywords[category] = keywords

# 6. 결과 출력
for category, keywords in category_keywords.items():
    print(f"\nCategory: {category}")
    print(f"Top 10 Keywords: {keywords}")